In [1]:
# Set main root 
from setup import setup_project_root
setup_project_root()

!hostname

languages = ['de', 'en', 'es', 'it', ]

node064


In [2]:
import torch.nn as nn

# Need to define arcitecture so that 

class LanguageDetector(nn.Module):
    def __init__(self):
        super().__init__()

        # 2 Convolution Layers
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)

        # Neuron Layers
        # Now apply linear layer --> Dimensions of input are 
        # Con1 
        # (1025, 216) --> (512, 108)
        # Con2
        # (512, 108) --> (256, 54)
        # Now we have 32 of these with respective filters applied
        self.fc1 = nn.Linear(32 * 256 * 54, 256)
        self.fc2 = nn.Linear(256, 32)
        self.fc3 = nn.Linear(32, len(languages))

        # Relu, Pool Function
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride = 2)


    def forward(self, x):

        # 2D Convolution Apllication
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))

        # Flatten Dimensions
        x = x.view(x.size(0), -1)

        # Dense Layers
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)

        return x

model = LanguageDetector()

In [3]:
import torch

model.load_state_dict(torch.load('models/model1.pth'))
test = torch.load('models/test.pt')

/tmp/ipykernel_2751477/2223399914.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('models/model1.pth'))
/tmp/ipykernel_2751477/222339991

In [4]:
from torch.utils.data import DataLoader

model.eval()

# Have to have small test size in order for server not to crash
testing = DataLoader(test, batch_size=16, shuffle=False)

In [5]:
y_pred = []
y_true = []

with torch.no_grad():
    for x_test, y_test in testing:
        outputs = model(x_test)

        _, predicted = torch.max(outputs, dim=1)

        y_pred.extend(predicted)
        y_true.extend(y_test)

In [6]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))

languages = ['de', 'en', 'es', 'it', ]

Accuracy: 0.40791666666666665

Classification Report:
               precision    recall  f1-score   support

           0       0.41      0.45      0.43       600
           1       0.42      0.41      0.42       600
           2       0.42      0.42      0.42       600
           3       0.38      0.35      0.37       600

    accuracy                           0.41      2400
   macro avg       0.41      0.41      0.41      2400
weighted avg       0.41      0.41      0.41      2400


Confusion Matrix:
 [[268 100  91 141]
 [156 249 109  86]
 [ 95 137 250 118]
 [138 105 145 212]]
